In [ ]:
import os
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import yt_dlp
import librosa
import numpy as np
import logging

# Setup basic logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

load_dotenv()

def get_audio_array(track_id, duration=30, sr=44100):
    """
    Get audio as numpy array from Spotify track ID using YouTube.
    
    Args:
        track_id: Spotify track ID
        duration: Duration in seconds (default 30)
        sr: Sample rate (default 22050 Hz)
    
    Returns:
        numpy array of audio samples, or None if failed
    """
    # Initialize Spotify client
    try:
        auth_manager = SpotifyClientCredentials(
            client_id=os.getenv("SPOTIFY_CLIENT_ID"),
            client_secret=os.getenv("SPOTIFY_CLIENT_SECRET")
        )
        sp = spotipy.Spotify(auth_manager=auth_manager)
        
        # Get track info from Spotify
        track = sp.track(track_id)
        track_name = track['name']
        artist_name = track['artists'][0]['name']
    except Exception as e:
        logger.error(f"Error fetching Spotify track {track_id}: {e}")
        return None
    
    # YouTube download options
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'temp_audio.%(ext)s',
        'quiet': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }],
    }
    
    # Download audio from YouTube
    search_query = f"{track_name} {artist_name} official audio"
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.extract_info(f"ytsearch1:{search_query}", download=True)
        
        # Load audio to numpy array
        audio, _ = librosa.load('temp_audio.wav', duration=duration, sr=sr, mono=True)
        
        # Clean up temp file
        # try:
        #     os.remove('temp_audio.wav')
        # except:
        #     pass
            
        logger.info(f"Successfully fetched audio for {track_name} by {artist_name}")
        return audio
        
    except Exception as e:
        logger.error(f"Error downloading audio for {track_id}: {e}")
        try:
            os.remove('temp_audio.wav')
        except:
            pass
        return None

# Example usage
if __name__ == "__main__":
    track_id = "2ChOqzPoYF5je9mfXtJrkW"  # Example Spotify track ID: CMYK by Ear
    audio = get_audio_array(track_id)
    if audio is not None:
        print(f"Audio shape: {audio.shape}, Type: {audio.dtype}")




INFO:__main__:Successfully fetched audio for CMYK by ear


Audio shape: (1323000,), Type: float32


: 